# Library/package install and code setup

In [1]:
#install for statistics functionality
!pip install pingouin

In [1]:
#import all other necessary libraries/packages
import pandas as pd
import numpy as np
from itertools import compress
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import h5py 
from scipy import interpolate
from scipy.io import loadmat
from scipy import signal
import random
import statsmodels.api as sm
from tqdm import tqdm
import pingouin as pg
tqdm.pandas()

import mne 
from mne_nirs.channels import picks_pair_to_idx
from mne.preprocessing.nirs import optical_density, beer_lambert_law, scalp_coupling_index, temporal_derivative_distribution_repair
from mne_nirs.signal_enhancement import (enhance_negative_correlation, short_channel_regression)
from mne_bids import write_raw_bids, BIDSPath, read_raw_bids
from mne_nirs.experimental_design import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
from mne_nirs.statistics import run_glm
from mne_nirs.channels import get_long_channels, get_short_channels

import scipy.signal as scs

C:\Users\vsinf\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# Aux signal filtering function

In [2]:
from scipy.signal import butter, filtfilt
def lowpass_filter(data, cutoff, fs, order=4):
    nyquist = 0.5 * fs  # Nyquist frequency
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Time lag selection function

In [3]:
# find_corr definition
# this will be used to find the optimal time lag for the physiological signals

def find_corr(col,long,aux):
    # print(f'FINDING CORRELATION WITH {aux_dict[n]} data...')
    nirs = long[col].to_numpy()

    # contain the cross correlation lags to be between -10 and 10
    correlation = np.correlate(aux - np.mean(aux), nirs - np.mean(nirs), mode='full')
    correlation = correlation[len(correlation)//2:len(correlation)//2+31]
    lags = signal.correlation_lags(len(aux), len(nirs), mode="full")

    # crop the lags to only include those between -10 and 10
    lags = lags[(lags >= 0) & (lags <= 30)]

    tlag = int(lags[np.argmax(abs(correlation))])
    # print(f'tlag is {tlag}')
    cross_corr = correlation[tlag]

    # SANITY CHECK
    # # plot the correlation for the given time lags
    
    # plt.figure()
    # plt.plot(lags, correlation)
    # plt.show()

    return tlag, cross_corr

# Canonical Correlation Analysis (CCA) function

In [4]:
from sklearn.cross_decomposition import CCA
def get_CCA(aux, fnirs):
    #print("aux:")
    #display(aux)
    fnirs = fnirs.iloc[:, 1:]
    #tolerance based on literature value (von Luhmann, 2020), n_components = 1 because of 1 fNIRS signal
    cca = CCA(n_components=1, tol=0.3) 
    cca.fit(aux, fnirs)
    aux_c = cca.transform(aux)
    #print('aux_c:')
    #print(aux_c)
    return pd.DataFrame(aux_c)
    

# Main function: establishes 6 correction models and runs GLM on each

In [7]:
ccaZ = pd.DataFrame()
#individual analysis function definition 
def individual_analysis_wings(bids_path, ID, auxs, region, verbose=False):
    raw_intensity = mne.io.read_raw_snirf(fname=bids_path, verbose=False, optode_frame="mri")
    
    # Delete annotation labeled 15, as these just signify the start and end of experiment.    
    raw_intensity.annotations.delete(raw_intensity.annotations.description == '15')
    raw_intensity.annotations.rename({'1': 'Speech',
                                      '2': 'Control'})
    raw_intensity.annotations.set_durations(5)

    # Convert raw light intensity signal to optical density and resample
    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
    
    # SCI thresholds; remove bad channels
    sci = scalp_coupling_index(raw_od, h_freq=1.35, h_trans_bandwidth=0.1)
    raw_od.info['bads'] = list(compress(raw_od.ch_names, sci < 0.7))
    raw_od.interpolate_bads()
    raw_od.resample(0.6)
    
    #convert optical densities to hemoglobin concentrations
    raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=.1)

        
    # define the source detector pairs for each ROI (this is specific to one's montage)
    Left_IFG = [[1,1], [2,1], [3,1], [4,1], [3,2], [4,2]]
    Left_HG = [[5,3], [5,4], [5,5], [6,3], [6,4], [6,5]]
    Left_PT = [[6,6], [6,7], [7,5], [7,7],  [8,6], [8,7]]
    Right_IFG = [[9,8], [10,8], [11,8], [11,9], [12,8], [12,9]]
    Right_HG = [[13,10], [13,11], [13,12], [14,10], [14,11], [14,12]]
    Right_PT = [[14,13], [14,14], [15,12], [15,14], [16,13], [16,14]]

    # Name ROIs
    rois = dict(
        Left_IFG = picks_pair_to_idx(raw_haemo, Left_IFG),
        Left_HG = picks_pair_to_idx(raw_haemo, Left_HG),
        Left_PT = picks_pair_to_idx(raw_haemo, Left_PT),
        Right_IFG = picks_pair_to_idx(raw_haemo, Right_IFG),
        Right_HG = picks_pair_to_idx(raw_haemo, Right_HG),
        Right_PT = picks_pair_to_idx(raw_haemo, Right_PT)
        )

    dat = h5py.File(bids_path, 'r')   

    ################################################################################
    ##############################   TCCA   ########################################
    ################################################################################
    
    # DEFINING THE DIFFERENT CORRECTION MODELS
    # wings/wingz -> ss + physio + lag (wingz with a z just to more easily differentiate from word "wings" in script)
    # no wings -> ss
    # no lag -> ss + physio
    # no ss -> no correction
    #no ss wings -> physio
    #ccaZ -> ss + tCCA
    
    # reset dataframes for each subject
    corr_df = pd.DataFrame()
    df = {}

    for i in range(6):
        df[i] = pd.DataFrame()

    # df dictionary ket: 
    # 0 = wings/z, 1 = no wings, 2 = no lag, 3 = no ss, 4 = no ss wings, 5 = ss + tCCA
    
    wingz = pd.DataFrame() 
    nolag = pd.DataFrame()
    short = get_short_channels(raw_haemo)

    #   pick the channels for the specified region ROI
    picks = rois[region]
    raw_haemo.pick(picks=picks)

    ## STORE THE AUX AND NIRS DATA FOR EACH TIME LAG
    
    for col in raw_haemo.to_data_frame().columns:
        # columns represent channels in the region of interest, 
        # this loop goes through each channel and creates a glm for each channel with specified wings data
        if col != 'time' and col.find('data') and col.find('hbr') == -1:
            wingz = pd.DataFrame()
            for n in auxs:
                # collect the aux data and timescale
                aux = np.array(dat.get(f'nirs/aux{n}/dataTimeSeries'))
                aux_time = np.array(dat.get(f'nirs/aux{n}/time'))
                fqs = np.argmax(aux_time > 1)   # samples of aux data per second (frequency of aux data)
                
                
                #if running all models but tCCA, just filter GSR (uncomment below code and comment tCCA filter line):
#                 if n==22:
#                      filtered_aux = lowpass_filter(aux, cutoff=5, fs=fqs, order=4)
#                 else:
#                     filtered_aux = aux

                #if running just tCCA, filter all aux signals:
                filtered_aux = lowpass_filter(aux, cutoff=0.5, fs=fqs, order=4)
        
                #convert to z-score
                mean_aux = np.mean(filtered_aux)
                std_aux = np.std(filtered_aux)
                z_scored_aux = (filtered_aux - mean_aux) / std_aux
                
                # match the filtered aux data timescale to the nirs data timescale
                aux_data_interp = interpolate.interp1d(aux_time, z_scored_aux, axis=0, bounds_error=False, fill_value='extrapolate')
                filtered_matched_aux = aux_data_interp(raw_haemo.times)

                # add aux data to the nolag dataframe
                nolag[aux_dict[n]] = filtered_matched_aux

                # use cross correlation to find the optimal time lag for each channel/aux pair and its correlation value
                tlag, cross_cor = find_corr(col, raw_haemo.to_data_frame(),filtered_matched_aux)
                
                # ensure that time lag is reasonable
                if tlag <= 30 and tlag >= 0:
                    # create a data frame to store the correlation data
                    lcols = [tlag, cross_cor, col, aux_dict[n], ID]
                    lcols = pd.DataFrame(lcols).T
                    corr_df = pd.concat([corr_df, lcols], ignore_index=True, axis=0)    # name the columns in the correlation data frame                    

                    # NOTE: uncomment these print statements to check aux data is being lagged correctly
                    # print(f'sps: {sps}')
                    # print(f'aux_time at sps: {aux_time[sps]}')

                    stl = int((fqs * tlag)) # samples of aux in the time lag

                    # print(f'stl: {stl}')
                    # print(f'aux_time at tlag = -10: {aux_time[sps * 10]}')
                    # shift the aux data by the time lag
                    # print(f'first auxes before rolling: {aux[stl: stl+ 10]}')

                    aux = np.roll(aux, stl)
                    
                    # print(f'first auxes after rolling: {aux[:10]}')

                    # match the aux data timescale to the nirs data timescale
                    aux_data_interp = interpolate.interp1d(aux_time, aux, axis=0, bounds_error=False, fill_value='extrapolate')
                    filtered_matched_aux = aux_data_interp(raw_haemo.times)
                    
                    wingz = pd.concat([wingz, pd.DataFrame(filtered_matched_aux)], axis=1)
            
            fnirs = raw_haemo.to_data_frame()
            #sanity print statements: 
            #display(wingz)
            #display(fnirs)
            
            #this is the tCCA correction model:
            ccaZ = get_CCA(wingz, fnirs)
            #print('ccaZ.shape:')
            #print(ccaZ.shape) 
            ccaZ.columns = ["CCA"]
            #display(ccaZ)
    ################################################################################
    ##########################   GLM ANALYSIS   ####################################
    ################################################################################            
    
            # create a design matrix based on the original nirs data
            design_matrix = make_first_level_design_matrix(raw_haemo,
                                                            drift_model='cosine',
                                                            high_pass=0.015,  # Must be specified per experiment
                                                            hrf_model='spm',
                                                            stim_dur=3)# Must be specified per experiment
            
            # add short channels to the design matrix
            design_matrix["ShortHbO"] = np.mean(short.copy().pick(
                                            picks="hbo").get_data(), axis=0)

            design_matrix["ShortHbR"] = np.mean(short.copy().pick(
                                            picks="hbr").get_data(), axis=0)

            # reset the index of the aux data to match the design matrix to prepare for concatenation
            wingz.index = design_matrix.index
            ccaZ.index = design_matrix.index

            nolag.index = design_matrix.index
            
            # add the aux data to the design matrix
            design_matrix_wings = pd.concat([design_matrix, wingz], axis=1)
            design_matrix_ccaZ = pd.concat([design_matrix, ccaZ], axis=1)

            design_matrix_nolag = pd.concat([design_matrix, nolag], axis=1)

            design_matrix_noss = design_matrix.copy()
            design_matrix_noss.drop(columns=['ShortHbO', 'ShortHbR'], inplace=True)

            design_matrix_nosswings = design_matrix_nolag.copy()
            design_matrix_nosswings.drop(columns=['ShortHbO', 'ShortHbR'], inplace=True)

            # run the glms 
            glm_est = run_glm(raw_haemo, design_matrix)
            glm_est_ccaZ = run_glm(raw_haemo, design_matrix_ccaZ)
            glm_est_wings = run_glm(raw_haemo, design_matrix_wings)
            glm_est_nolag = run_glm(raw_haemo, design_matrix_nolag)
            glm_est_noss = run_glm(raw_haemo, design_matrix_noss)
            glm_est_nosswings = run_glm(raw_haemo, design_matrix_nosswings)

            # store the glm data in dataframes
            temp_nowings = glm_est.to_dataframe()
            temp_nowings["ID"] = ID  # Add the participant ID to the dataframe

            temp_wings = glm_est_wings.to_dataframe()
            temp_wings["ID"] = ID  # Add the participant ID to the dataframe
            
            temp_ccaZ = glm_est_ccaZ.to_dataframe()
            temp_ccaZ["ID"] = ID  # Add the participant ID to the dataframe
            
            temp_nolag = glm_est_nolag.to_dataframe()
            temp_nolag["ID"] = ID  # Add the participant ID to the dataframe

            temp_noss = glm_est_noss.to_dataframe()
            temp_noss["ID"] = ID  # Add the participant ID to the dataframe

            temp_nosswings = glm_est_nosswings.to_dataframe()
            temp_nosswings["ID"] = ID  # Add the participant ID to the dataframe

            # add the glm dataframes to the main dataframes
            df[0] = pd.concat([df[0], temp_wings], axis=0)
            df[1] = pd.concat([df[1], temp_nowings], axis=0)
            df[2] = pd.concat([df[2], temp_nolag], axis=0)
            df[3] = pd.concat([df[3], temp_noss], axis=0)
            df[4] = pd.concat([df[4], temp_nosswings], axis=0)
            df[5] = pd.concat([df[5], temp_ccaZ], axis=0)

            # sanity check to make sure the dataframes are as expected
            # display(df[0])
            # display(df[1])
            # display(df[2])

            # drop duplicate rows if they exist (they shouldn't)
            df[0].drop_duplicates(subset=['ch_name', 'Condition'], inplace=True)
            df[1].drop_duplicates(subset=['ch_name', 'Condition'], inplace=True)
            df[2].drop_duplicates(subset=['ch_name', 'Condition'], inplace=True)
            df[3].drop_duplicates(subset=['ch_name', 'Condition'], inplace=True)
            df[4].drop_duplicates(subset=['ch_name', 'Condition'], inplace=True)
            df[5].drop_duplicates(subset=['ch_name', 'Condition'], inplace=True)

            # SANITY CHECK: make sure the dataframes are all the same length
            # print(f'df[0] shape: {df[0].shape}')
            # print(f'df[1] shape: {df[1].shape}')
            # print(f'df[2] shape: {df[2].shape}')

    df[0].reset_index(drop=False, inplace=True)
    df[1].reset_index(drop=False, inplace=True)
    df[2].reset_index(drop=False, inplace=True)
    df[3].reset_index(drop=False, inplace=True)
    df[4].reset_index(drop=False, inplace=True)
    df[5].reset_index(drop=False, inplace=True)


    corr_df.columns = ['tlag', 'cross_cor', 'SD pair', 'aux', 'ID']

    roi_df = {}
    for i in range(5):
        roi_df[i] = pd.DataFrame()

    # find the descirptive stats for each condition w and w/o wings and lags
    # for df, roi in (df[0], df[1], df[2], df[3], df[4]), df[5], (roi_wings, roi_nowings, roi_nolag, roi_noss, roi_nosswings, roi_ccaZ):
    for i in range(6):
        orig_df = df[i].copy()
        temp_roi = pd.DataFrame()
        for chan in ['hbo', 'hbr']:
            chan_df = pd.DataFrame()
            # reset the frame to the original frame
            current_df = orig_df.copy()

            # select the rows with the specified chromophore
            current_df = current_df[current_df['ch_name'].str.contains(chan)]            
            temp_df = current_df[current_df['Condition'].isin(['Speech', 'Control'])]

            beta = temp_df.groupby('Condition')['theta'].mean()
            sd = temp_df.groupby('Condition')['theta'].std()
            cv = sd / beta

            sse = temp_df.groupby('Condition')['se'].progress_apply(lambda x: np.sum(x**2))
            t2 = temp_df.groupby('Condition')['t'].progress_apply(lambda x: np.sum(x**2))
            n = temp_df.groupby('Condition').size()
            chan_df['se_manual'] = sd / np.sqrt(n)
            chan_df['se'] = temp_df.groupby(['Condition'])['theta'].std() / np.sqrt(temp_df.groupby(['Condition'])['theta'].count())
            chan_df['rmse (e-6)'] = np.sqrt(sse / n) * 1000000
            chan_df['beta'] = beta 
            chan_df['CV'] = cv
            chan_df['F'] = np.sqrt(t2 / n)
            chan_df['df'] = current_df.groupby('Condition')['df'].mean()
            chan_df['ID'] = ID
            chan_df['Chroma'] = chan
            chan_df['Condition'] = ['Control', 'Speech']

            temp_roi = pd.concat([temp_roi, chan_df], axis=0)

        roi_df[i] = temp_roi
    #return desired values for glm analyses
    return df[0], df[1], df[2], df[3], df[4], df[5], roi_df[0], roi_df[1], roi_df[2], roi_df[3], roi_df[4], roi_df[5], corr_df, chan_df, temp_df, design_matrix_ccaZ

# Raw data loading, function calling, GLM results storing

In [8]:
#will have to run this twice: once for all models except tCCA, and one for gettingthe tCCA results (different filtering - look in above cell for what to uncomment)

df_roi = pd.DataFrame()  # To store region of interest results 
df_wings = pd.DataFrame()  # To store region of interest results
df_nolag = pd.DataFrame()  # To store region of interest results
df_nowings = pd.DataFrame()  # To store region of interest results
df_noshort = pd.DataFrame()  # To store region of interest results
df_noss = pd.DataFrame() # To store region of interest results
df_nosswings = pd.DataFrame() # To store region of interest results
df_ccaZ = pd.DataFrame() # To store region of interest results

temp_df = pd.DataFrame()
tlags_df = pd.DataFrame()
sub_df = {}

all_rois = pd.DataFrame()

chans = [
    'accelerometer_1_x', 'accelerometer_1_y', 'accelerometer_1_z',
    'gyroscope_1_x', 'gyroscope_1_y', 'gyroscope_1_z',
    'accelerometer_2_x', 'accelerometer_2_y', 'accelerometer_2_z',
    'gyroscope_2_x', 'gyroscope_2_y', 'gyroscope_2_z',
    'Resp.', 'PPG', 'SpO2', 'HR', 'GSR', 'Temp.'
]


#auxs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 18, 19, 20, 21, 22, 23]
auxs = [18, 19, 20, 21, 22, 23] #just want the pertinent aux signals that we have been focused on

# auxs = [18]
nsubs = 10

# create a dictionary to match the aux channel number to the aux channel name (for 18-23)
aux_dict = {}
for i in auxs:
    if i < 13:
        aux_dict[i] = chans[i-1]
    else:
        aux_dict[i] = chans[i-6]

# time lags are shifts
# NOTE: a time lag of -5 means shifting the auxilary signal 5 seconds to the left (negative direction); 
# tlag of 5 means shifting the hemoglobin data 5 seconds to the left (negative direction)

for region in ['Left_PT', 'Left_HG', 'Left_IFG', 'Right_PT', 'Right_HG', 'Right_IFG']:
# for region in ['Left_PT']:
    print(f'Analyzing {region}...')
    roi_wings = pd.DataFrame()
    roi_nowings = pd.DataFrame()
    roi_nolag = pd.DataFrame()
    roi_noss = pd.DataFrame()
    roi_nosswings = pd.DataFrame()
    roi_ccaZ = pd.DataFrame()
    corr_df = pd.DataFrame()

    #for sub in range(1, nsubs+1):  # change range values to specify the number of subject recordings to be analyzed     
    for sub in range(1,11):
    #for sub in [3]:    
    # dm = pd.DataFrame()
        ID = '%02d' % sub
        bids_path = BIDSPath(
                subject="%02d" % sub,
               # task="wings",
                session='01',
                datatype="nirs",
               # suffix='nirs',
                root=r"C:\Users\vsinf\Box\BRAiN Lab\current projects\repeatability-project\sourcedata",
                extension=".snirf"
            )

        sub_wings, sub_nowings, sub_nolag, sub_noss, sub_nosswings, sub_ccaZ, sub_roiw, sub_roinw, sub_roinl, sub_roinss, sub_roinosswings, sub_roiccaZ, sub_corrs, chan_df, temp_df, design_matrix_ccaZ = individual_analysis_wings(bids_path, ID, auxs, region)
       
    #PRINT STATEMENTS FOR SANITY CHECK:
#         print('sub_wings')
#         print(sub_wings)
        
#         print('sub_nowings')
#         print(sub_nowings)
        
#         print('sub_nolag')
#         print(sub_nolag)
       
#         print('sub_noss')        
#         print(sub_noss)

        #print('sub_nosswings')
        #print(sub_nosswings)

        
        df_wings = pd.concat([df_wings, sub_wings], ignore_index=True)
        df_nowings = pd.concat([df_nowings, sub_nowings], ignore_index=True)
        df_nolag = pd.concat([df_nolag, sub_nolag], ignore_index=True)
        df_noss = pd.concat([df_noss, sub_noss], ignore_index=True)
        df_nosswings = pd.concat([df_nosswings, sub_nosswings], ignore_index=True)
        df_ccaZ = pd.concat([df_ccaZ, sub_ccaZ], ignore_index=True)
        
        
        roi_wings = pd.concat([roi_wings, sub_roiw], ignore_index=True)
        roi_nowings = pd.concat([roi_nowings, sub_roinw], ignore_index=True)
        roi_nolag = pd.concat([roi_nolag, sub_roinl], ignore_index=True)
        roi_noss = pd.concat([roi_noss, sub_roinss], ignore_index=True)
        roi_nosswings = pd.concat([roi_nosswings, sub_roinosswings], ignore_index=True)
        roi_ccaZ = pd.concat([roi_ccaZ, sub_roiccaZ], ignore_index=True)

        sub_corrs['ROI'] = region
        corr_df = pd.concat([corr_df, sub_corrs], ignore_index=True)

    roi_wings['Wings'] = 'Lagged Wings'
    roi_nolag['Wings'] = 'Unlagged Wings'
    roi_nowings['Wings'] = 'No Wings'
    roi_noss['Wings'] = 'No SS No Wings'
    roi_nosswings['Wings'] = 'No SS w/ Wings'
    roi_ccaZ['Wings'] = 'CCA'

    for df in [roi_wings, roi_nowings, roi_nolag, roi_noss, roi_nosswings, roi_ccaZ]:
        df['ROI'] = region

    # add the roi data to the all_rois dataframe
    all_rois = pd.concat([all_rois, roi_wings], axis=0, ignore_index=True)
    all_rois = pd.concat([all_rois, roi_nolag], axis=0, ignore_index=True)
    all_rois = pd.concat([all_rois, roi_nowings], axis=0, ignore_index=True)
    all_rois = pd.concat([all_rois, roi_noss], axis=0, ignore_index=True)
    all_rois = pd.concat([all_rois, roi_nosswings], axis=0, ignore_index=True)
    all_rois = pd.concat([all_rois, roi_ccaZ], axis=0, ignore_index=True)
corr_df_data = corr_df.copy()

Analyzing Left_PT...
Reading 0 ... 5829  =      0.000 ...  1145.981 secs...
Setting channel interpolation method to {'fnirs': 'nearest'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 97.2 mm


C:\Users\vsinf\AppData\Local\Temp\ipykernel_10808\4097024209.py:18: RuntimeWarning: (X, Y) fit (0.1, -20.9) more than 20 mm from head frame origin
  raw_od.interpolate_bads()
100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1020.64it/s]


Reading 0 ... 6250  =      0.000 ...  1228.750 secs...
Setting channel interpolation method to {'fnirs': 'nearest'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 97.2 mm


C:\Users\vsinf\AppData\Local\Temp\ipykernel_10808\4097024209.py:18: RuntimeWarning: (X, Y) fit (0.1, -20.9) more than 20 mm from head frame origin
  raw_od.interpolate_bads()
100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1512.01it/s]


Reading 0 ... 6022  =      0.000 ...  1183.925 secs...
Setting channel interpolation method to {'fnirs': 'nearest'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 97.2 mm


...

C:\Users\vsinf\AppData\Local\Temp\ipykernel_10808\4097024209.py:18: RuntimeWarning: (X, Y) fit (0.1, -20.9) more than 20 mm from head frame origin
  raw_od.interpolate_bads()
100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


Reading 0 ... 5794  =      0.000 ...  1139.100 secs...
Setting channel interpolation method to {'fnirs': 'nearest'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 97.2 mm


C:\Users\vsinf\AppData\Local\Temp\ipykernel_10808\4097024209.py:18: RuntimeWarning: (X, Y) fit (0.1, -20.9) more than 20 mm from head frame origin
  raw_od.interpolate_bads()
100%|████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<?, ?it/s]


Reading 0 ... 5872  =      0.000 ...  1154.435 secs...
Setting channel interpolation method to {'fnirs': 'nearest'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 97.2 mm


C:\Users\vsinf\AppData\Local\Temp\ipykernel_10808\4097024209.py:18: RuntimeWarning: (X, Y) fit (0.1, -20.9) more than 20 mm from head frame origin
  raw_od.interpolate_bads()
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 127.96it/s]


In [68]:
#just putting everything into a csv file
all_rois.to_csv('all_rois_allsubs.csv')

# GLM results formatting

In [9]:
#look at full GLM results dataset across all ROIs
all_rois

,se_manual,se,rmse (e-6),beta,CV,F,df,ID,Chroma,Condition,Wings,ROI
0,7.495085e-07,7.495085e-07,2.837578,-8.385998e-08,-21.892603,0.606616,45.0,01,hbo,Control,Lagged Wings,Left_PT
1,3.695299e-06,3.695299e-06,2.854211,2.023514e-05,0.447321,7.313012,45.0,01,hbo,Speech,Lagged Wings,Left_PT
2,5.046167e-07,5.046167e-07,1.205221,-1.311436e-06,-0.942519,1.239675,45.0,01,hbr,Control,Lagged Wings,Left_PT
3,1.849140e-06,1.849140e-06,1.212448,-2.285267e-06,-1.982022,3.874661,45.0,01,hbr,Speech,Lagged Wings,Left_PT
4,3.381604e-07,3.381604e-07,2.399777,-1.022180e-06,-0.810347,0.531431,47.0,02,hbo,Control,Lagged Wings,Left_PT
...,...,...,...,...,...,...,...,...,...,...,...,...
1435,2.571916e-06,2.571916e-06,7.217375,-2.959600e-06,-2.128626,0.717423,40.0,09,hbr,Speech,CCA,Right_IFG
1436,4.766625e-06,4.766625e-06,12.192551,1.134956e-05,1.028745,1.084454,40.0,10,hbo,Control,CCA,Right_IFG
1437,1.129732e-05,1.129732e-05,12.268104,3.160467e-05,0.875588,3.147416,40.0,10,hbo,Speech,CCA,Right_IFG
1438,1.052130e-06,1.052130e-06,5.011287,1.581614e-06,1.629464,0.681058,40.0,10,hbr,Control,CCA,Right_IFG


In [10]:
# create overall condition with correction type, speech/control, hbo/hbr across all ROIs
all_rois['Overall Condition'] = all_rois['Wings'] + ' ' + all_rois['Condition'] + ' ' + all_rois['Chroma']

In [11]:
#SANITY CHECK: verifying that all Overall Conditions are present 
display(all_rois[all_rois['Overall Condition'].isna()])

# print the number of ratings for each condition
print(f'Number of ratings for each condition: \n{all_rois.groupby("Overall Condition").size()}')

,se_manual,se,rmse (e-6),beta,CV,F,df,ID,Chroma,Condition,Wings,ROI,Overall Condition


Number of ratings for each condition: 
Overall Condition
CCA Control hbo               60
CCA Control hbr               60
CCA Speech hbo                60
CCA Speech hbr                60
Lagged Wings Control hbo      60
Lagged Wings Control hbr      60
Lagged Wings Speech hbo       60
Lagged Wings Speech hbr       60
No SS No Wings Control hbo    60
No SS No Wings Control hbr    60
No SS No Wings Speech hbo     60
No SS No Wings Speech hbr     60
No SS w/ Wings Control hbo    60
No SS w/ Wings Control hbr    60
No SS w/ Wings Speech hbo     60
No SS w/ Wings Speech hbr     60
No Wings Control hbo          60
No Wings Control hbr          60
No Wings Speech hbo           60
No Wings Speech hbr           60
Unlagged Wings Control hbo    60
Unlagged Wings Control hbr    60
Unlagged Wings Speech hbo     60
Unlagged Wings Speech hbr     60
dtype: int64


In [12]:
# Visualize expanded all_rois (with new Overall Condition column)
all_rois

,se_manual,se,rmse (e-6),beta,CV,F,df,ID,Chroma,Condition,Wings,ROI,Overall Condition
0,7.495085e-07,7.495085e-07,2.837578,-8.385998e-08,-21.892603,0.606616,45.0,01,hbo,Control,Lagged Wings,Left_PT,Lagged Wings Control hbo
1,3.695299e-06,3.695299e-06,2.854211,2.023514e-05,0.447321,7.313012,45.0,01,hbo,Speech,Lagged Wings,Left_PT,Lagged Wings Speech hbo
2,5.046167e-07,5.046167e-07,1.205221,-1.311436e-06,-0.942519,1.239675,45.0,01,hbr,Control,Lagged Wings,Left_PT,Lagged Wings Control hbr
3,1.849140e-06,1.849140e-06,1.212448,-2.285267e-06,-1.982022,3.874661,45.0,01,hbr,Speech,Lagged Wings,Left_PT,Lagged Wings Speech hbr
4,3.381604e-07,3.381604e-07,2.399777,-1.022180e-06,-0.810347,0.531431,47.0,02,hbo,Control,Lagged Wings,Left_PT,Lagged Wings Control hbo
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,2.571916e-06,2.571916e-06,7.217375,-2.959600e-06,-2.128626,0.717423,40.0,09,hbr,Speech,CCA,Right_IFG,CCA Speech hbr
1436,4.766625e-06,4.766625e-06,12.192551,1.134956e-05,1.028745,1.084454,40.0,10,hbo,Control,CCA,Right_IFG,CCA Control hbo
1437,1.129732e-05,1.129732e-05,12.268104,3.160467e-05,0.875588,3.147416,40.0,10,hbo,Speech,CCA,Right_IFG,CCA Speech hbo
1438,1.052130e-06,1.052130e-06,5.011287,1.581614e-06,1.629464,0.681058,40.0,10,hbr,Control,CCA,Right_IFG,CCA Control hbr


# Dataframe filtering to specific ROI, chroma, correction model, and condition

In [13]:
import pandas as pd

# Filtering the DataFrame for 'pt' in 'ROI' (case-insensitive)
filtered_rois = all_rois[
    (all_rois['ROI'].str.contains('pt', case=False)) & # change depending on ROI desired 
    (all_rois['Chroma'] == 'hbo') & # change depending on chroma desired 
    (all_rois['Wings'] == 'CCA') & # change depending on correction model desired
    (all_rois['Condition'] == 'Speech') # we are primarily interested in speech betas
]

# Display the filtered DataFrame
filtered_rois


,se_manual,se,rmse (e-6),beta,CV,F,df,ID,Chroma,Condition,Wings,ROI,Overall Condition
201,3.928821e-06,3.928821e-06,2.841343,0.000020,0.476512,7.397389,40.0,01,hbo,Speech,CCA,Left_PT,CCA Speech hbo
205,8.685472e-07,8.685472e-07,2.422488,0.000012,0.184921,4.905852,42.0,02,hbo,Speech,CCA,Left_PT,CCA Speech hbo
209,1.711829e-06,1.711829e-06,2.844561,0.000009,0.486636,3.187180,41.0,03,hbo,Speech,CCA,Left_PT,CCA Speech hbo
213,1.381908e-06,1.381908e-06,2.808008,0.000008,0.440828,3.369179,42.0,04,hbo,Speech,CCA,Left_PT,CCA Speech hbo
217,2.302034e-06,2.302034e-06,2.901390,0.000003,1.865054,1.996271,41.0,05,hbo,Speech,CCA,Left_PT,CCA Speech hbo
221,3.694566e-06,3.694566e-06,3.988080,0.000020,0.443121,5.228256,42.0,06,hbo,Speech,CCA,Left_PT,CCA Speech hbo
225,1.442976e-06,1.442976e-06,2.845394,0.000005,0.780302,2.115936,41.0,07,hbo,Speech,CCA,Left_PT,CCA Speech hbo
229,1.840351e-06,1.840351e-06,3.219737,0.000003,1.489078,1.643930,40.0,08,hbo,Speech,CCA,Left_PT,CCA Speech hbo
233,1.691018e-06,1.691018e-06,3.732110,0.000005,0.891522,1.863192,40.0,09,hbo,Speech,CCA,Left_PT,CCA Speech hbo
237,1.320353e-06,1.320353e-06,3.085027,0.000007,0.485237,2.608656,40.0,10,hbo,Speech,CCA,Left_PT,CCA Speech hbo


In [14]:
#streamlining format (this can be tailored towards your preferences)
filtered_rois.rename(columns={'ROI': 'ROI_detail'}, inplace=True)
filtered_rois['hemi'] = filtered_rois['ROI_detail'].apply(lambda x: x[0])
filtered_rois['ROI'] = filtered_rois['ROI_detail'].apply(lambda x: x.split('_')[1].lower())
filtered_rois['beta'] = filtered_rois['beta'] * 1e6 #for easier plotting 

filtered_rois['ID'] = filtered_rois['ID'].astype(int)

# Function to determine the day
def determine_day(id_value):
    return (id_value + 1) // 2

# Apply the function to create the 'day' column
filtered_rois['day'] = filtered_rois['ID'].apply(determine_day)
filtered_rois.rename(columns={'ID': 'sub'}, inplace=True)

# Function to determine the time
def determine_time(id_value):
    return 'AM' if id_value % 2 != 0 else 'PM'

# Apply the function to create the 'time' column
filtered_rois['time'] = filtered_rois['sub'].apply(determine_time)

filtered_rois

C:\Users\vsinf\AppData\Local\Temp\ipykernel_10808\29906423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rois.rename(columns={'ROI': 'ROI_detail'}, inplace=True)
C:\Users\vsinf\AppData\Local\Temp\ipykernel_10808\29906423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rois['hemi'] = filtered_rois['ROI_detail'].apply(lambda x: x[0])
C:\Users\vsinf\AppData\Local\Temp\ipykernel_10808\29906423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

,se_manual,se,rmse (e-6),beta,CV,F,df,sub,Chroma,Condition,Wings,ROI_detail,Overall Condition,hemi,ROI,day,time
201,3.928821e-06,3.928821e-06,2.841343,20.195943,0.476512,7.397389,40.0,1,hbo,Speech,CCA,Left_PT,CCA Speech hbo,L,pt,1,AM
205,8.685472e-07,8.685472e-07,2.422488,11.504921,0.184921,4.905852,42.0,2,hbo,Speech,CCA,Left_PT,CCA Speech hbo,L,pt,1,PM
209,1.711829e-06,1.711829e-06,2.844561,8.616516,0.486636,3.187180,41.0,3,hbo,Speech,CCA,Left_PT,CCA Speech hbo,L,pt,2,AM
213,1.381908e-06,1.381908e-06,2.808008,7.678668,0.440828,3.369179,42.0,4,hbo,Speech,CCA,Left_PT,CCA Speech hbo,L,pt,2,PM
217,2.302034e-06,2.302034e-06,2.901390,3.023402,1.865054,1.996271,41.0,5,hbo,Speech,CCA,Left_PT,CCA Speech hbo,L,pt,3,AM
221,3.694566e-06,3.694566e-06,3.988080,20.422845,0.443121,5.228256,42.0,6,hbo,Speech,CCA,Left_PT,CCA Speech hbo,L,pt,3,PM
225,1.442976e-06,1.442976e-06,2.845394,4.529728,0.780302,2.115936,41.0,7,hbo,Speech,CCA,Left_PT,CCA Speech hbo,L,pt,4,AM
229,1.840351e-06,1.840351e-06,3.219737,3.027324,1.489078,1.643930,40.0,8,hbo,Speech,CCA,Left_PT,CCA Speech hbo,L,pt,4,PM
233,1.691018e-06,1.691018e-06,3.732110,4.646134,0.891522,1.863192,40.0,9,hbo,Speech,CCA,Left_PT,CCA Speech hbo,L,pt,5,AM
237,1.320353e-06,1.320353e-06,3.085027,6.665174,0.485237,2.608656,40.0,10,hbo,Speech,CCA,Left_PT,CCA Speech hbo,L,pt,5,PM


# CSV file creation of each filtered dataset for storing and external plotting

In [32]:

#filtered_rois['se'] = filtered_rois['rmse (e-6)'] #se and rmse are not equivalent but in plotting, rmse is denoted as se, so this just allows for consistent labeling 

#filtered_rois.to_csv('all_rois_fromTCCA.csv')

#All csv file names:

#HBO
#filtered_rois.to_csv('TCCA_SSPhysioLag.csv')
#filtered_rois.to_csv('TCCA_SSPhysio.csv')
#filtered_rois.to_csv('TCCA_SS.csv')
#filtered_rois.to_csv('TCCA_nocorrection.csv')
#filtered_rois.to_csv('TCCA_Physio.csv')
filtered_rois.to_csv('TCCA_CCA.csv')

#IFG
#filtered_rois.to_csv('TCCA_SSPhysioLag_ifg.csv')
#filtered_rois.to_csv('TCCA_SSPhysio_ifg.csv')
#filtered_rois.to_csv('TCCA_SS_ifg.csv')
#filtered_rois.to_csv('TCCA_nocorrection_ifg.csv')
#filtered_rois.to_csv('TCCA_Physio_ifg2.csv')
#filtered_rois.to_csv('TCCA_CCA_ifg.csv')



#HBR
#filtered_rois.to_csv('TCCA_SSPhysioLag_hbr.csv')
#filtered_rois.to_csv('TCCA_SSPhysio_hbr.csv')
#filtered_rois.to_csv('TCCA_SS_hbr.csv')
#filtered_rois.to_csv('TCCA_nocorrection_hbr.csv')
#filtered_rois.to_csv('TCCA_Physio_hbr.csv')
#filtered_rois.to_csv('TCCA_CCA_hbr.csv')





#IFG
#filtered_rois.to_csv('TCCA_SSPhysioLag_hbr_ifg.csv')
#filtered_rois.to_csv('TCCA_SSPhysio_hbr_ifg.csv')
#filtered_rois.to_csv('TCCA_SS_hbr_ifg.csv')
#filtered_rois.to_csv('TCCA_nocorrection_hbr_ifg.csv')
#filtered_rois.to_csv('TCCA_Physio_hbr_ifg.csv')
#filtered_rois.to_csv('TCCA_CCA_hbr_ifg.csv')



C:\Users\vsinf\AppData\Local\Temp\ipykernel_8840\1697491558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rois['se'] = filtered_rois['rmse (e-6)'] #se and rmse are not equivalent but in plotting, rmse is denoted as se, so this just allows for consistent labeling


# Formatting of FULL, unfiltered table of GLM results for all ROIs (including time of day, hemisphere, etc.)

In [15]:
#SANITY CHECK to make sure all rois look good
all_rois

,se_manual,se,rmse (e-6),beta,CV,F,df,ID,Chroma,Condition,Wings,ROI,Overall Condition
0,7.495085e-07,7.495085e-07,2.837578,-8.385998e-08,-21.892603,0.606616,45.0,01,hbo,Control,Lagged Wings,Left_PT,Lagged Wings Control hbo
1,3.695299e-06,3.695299e-06,2.854211,2.023514e-05,0.447321,7.313012,45.0,01,hbo,Speech,Lagged Wings,Left_PT,Lagged Wings Speech hbo
2,5.046167e-07,5.046167e-07,1.205221,-1.311436e-06,-0.942519,1.239675,45.0,01,hbr,Control,Lagged Wings,Left_PT,Lagged Wings Control hbr
3,1.849140e-06,1.849140e-06,1.212448,-2.285267e-06,-1.982022,3.874661,45.0,01,hbr,Speech,Lagged Wings,Left_PT,Lagged Wings Speech hbr
4,3.381604e-07,3.381604e-07,2.399777,-1.022180e-06,-0.810347,0.531431,47.0,02,hbo,Control,Lagged Wings,Left_PT,Lagged Wings Control hbo
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,2.571916e-06,2.571916e-06,7.217375,-2.959600e-06,-2.128626,0.717423,40.0,09,hbr,Speech,CCA,Right_IFG,CCA Speech hbr
1436,4.766625e-06,4.766625e-06,12.192551,1.134956e-05,1.028745,1.084454,40.0,10,hbo,Control,CCA,Right_IFG,CCA Control hbo
1437,1.129732e-05,1.129732e-05,12.268104,3.160467e-05,0.875588,3.147416,40.0,10,hbo,Speech,CCA,Right_IFG,CCA Speech hbo
1438,1.052130e-06,1.052130e-06,5.011287,1.581614e-06,1.629464,0.681058,40.0,10,hbr,Control,CCA,Right_IFG,CCA Control hbr


In [16]:
all_rois_ready = all_rois

# streamline formatting for all_rois
all_rois_ready.rename(columns={'ROI': 'ROI_detail'}, inplace=True)
all_rois_ready['hemi'] = all_rois_ready['ROI_detail'].apply(lambda x: x[0])
all_rois_ready['ROI'] = all_rois_ready['ROI_detail'].apply(lambda x: x.split('_')[1].lower())
all_rois_ready['beta'] = all_rois_ready['beta'] * 1e6

all_rois_ready['ID'] = all_rois_ready['ID'].astype(int)

# Function to determine the day
def determine_day(id_value):
    return (id_value + 1) // 2

# Apply the function to create the 'day' column
all_rois_ready['day'] = all_rois_ready['ID'].apply(determine_day)
all_rois_ready.rename(columns={'ID': 'sub'}, inplace=True)

# Function to determine the time
def determine_time(id_value):
    return 'AM' if id_value % 2 != 0 else 'PM'

# Apply the function to create the 'time' column
all_rois_ready['time'] = all_rois_ready['sub'].apply(determine_time)

all_rois_ready

# Can be saved as a csv if desired
#all_rois_ready.to_csv('all_rois_fullformat.csv') 

,se_manual,se,rmse (e-6),beta,CV,F,df,sub,Chroma,Condition,Wings,ROI_detail,Overall Condition,hemi,ROI,day,time
0,7.495085e-07,7.495085e-07,2.837578,-0.083860,-21.892603,0.606616,45.0,1,hbo,Control,Lagged Wings,Left_PT,Lagged Wings Control hbo,L,pt,1,AM
1,3.695299e-06,3.695299e-06,2.854211,20.235142,0.447321,7.313012,45.0,1,hbo,Speech,Lagged Wings,Left_PT,Lagged Wings Speech hbo,L,pt,1,AM
2,5.046167e-07,5.046167e-07,1.205221,-1.311436,-0.942519,1.239675,45.0,1,hbr,Control,Lagged Wings,Left_PT,Lagged Wings Control hbr,L,pt,1,AM
3,1.849140e-06,1.849140e-06,1.212448,-2.285267,-1.982022,3.874661,45.0,1,hbr,Speech,Lagged Wings,Left_PT,Lagged Wings Speech hbr,L,pt,1,AM
4,3.381604e-07,3.381604e-07,2.399777,-1.022180,-0.810347,0.531431,47.0,2,hbo,Control,Lagged Wings,Left_PT,Lagged Wings Control hbo,L,pt,1,PM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,2.571916e-06,2.571916e-06,7.217375,-2.959600,-2.128626,0.717423,40.0,9,hbr,Speech,CCA,Right_IFG,CCA Speech hbr,R,ifg,5,AM
1436,4.766625e-06,4.766625e-06,12.192551,11.349557,1.028745,1.084454,40.0,10,hbo,Control,CCA,Right_IFG,CCA Control hbo,R,ifg,5,PM
1437,1.129732e-05,1.129732e-05,12.268104,31.604667,0.875588,3.147416,40.0,10,hbo,Speech,CCA,Right_IFG,CCA Speech hbo,R,ifg,5,PM
1438,1.052130e-06,1.052130e-06,5.011287,1.581614,1.629464,0.681058,40.0,10,hbr,Control,CCA,Right_IFG,CCA Control hbr,R,ifg,5,PM
